In [47]:
from tensorflow import keras
import pandas as pd
import numpy as np
from keras import backend as K
import matplotlib.pyplot as plt

In [37]:
nFeat = 6
trainsize = 499889
intermediate_dim = 128
encoder_dim = 128
batch_size = 1024
latent_dim = 6
epochs = 240
eluvar = np.sqrt(1.55/intermediate_dim)

folder = "/global/u1/e/elham/anomaly_detection/AD-cBVAE/outputs/models/z6_smallModel_dymanicbeta_02-12-2022/"

In [38]:
decoder = keras.models.load_model(folder+'/model_cbvae_6var_m119.h5')

In [39]:
list_of_decoders = [decoder]

In [40]:
encoder = keras.models.load_model(folder+'/encoder_cbvae_6var_m119.h5')

In [41]:
list_of_encoders = [encoder]

## Load SR data

In [20]:
innerdata_train = np.load("./data/preprocessed_data_6var//innerdata_train_6var.npy")
innerdata_train = innerdata_train[innerdata_train[:,nFeat+1]==0]

In [21]:
# check number of total events & bkg in innerdata_train

pd_innerdata_train = pd.DataFrame(innerdata_train, columns=["mjj", "mjmin", "mjmax-mjmin", "tau21j1", "tau21j2", "tau32j1", "tau32j2", "label"])
print("number of events in innerdata_train:", len(pd_innerdata_train))
pd_innerdata_train_bkg = pd_innerdata_train[pd_innerdata_train["label"]==0]
print("number of events in innerdata with label=0 (bkg):", len(pd_innerdata_train_bkg))

number of events in innerdata_train: 59605
number of events in innerdata with label=0 (bkg): 59605


In [22]:
X_innerdata_train = innerdata_train[:,1:nFeat+1]
y_innerdata_train = innerdata_train[:,0]

print("shape of X_innerdata_train:", X_innerdata_train.shape)
print("shape of y_innerdata_train:", y_innerdata_train.shape)

shape of X_innerdata_train: (59605, 6)
shape of y_innerdata_train: (59605,)


In [23]:
train_features = ["mjmin", "mjmax_minus_mjmin", "tau21j1", "tau21j2", "tau32j1", "tau32j2"]
# train_features = ["mjmin"]
condition_features = ["mjj"]

nFeat = len(train_features)

## Load SB data

In [25]:
# create scaler

outerdata_train = np.load("./data/preprocessed_data_6var/outerdata_train_6var.npy")
outerdata_test = np.load("./data/preprocessed_data_6var/outerdata_test_6var.npy")
outerdata_train = outerdata_train[outerdata_train[:,nFeat+1]==0]
outerdata_test = outerdata_test[outerdata_test[:,nFeat+1]==0]

X_data_train_unscaled = outerdata_train[:,1:nFeat+1]
y_data_train_unscaled = outerdata_train[:,0]

X_data_test_unscaled = outerdata_test[:,1:nFeat+1]
y_data_test_unscaled = outerdata_test[:,0]

X_data_unscaled = np.concatenate((X_data_train_unscaled, X_data_test_unscaled), axis=0)
X_data = np.copy(X_data_unscaled)
y_data = np.concatenate((y_data_train_unscaled, y_data_test_unscaled), axis=0)

# scaler = MinMaxScaler((0,1)).fit(X_data)
# scaler_mjj = MinMaxScaler((0,1)).fit(y_data.reshape(-1,1))
x_max = np.empty(nFeat)
for i in range(0,X_data.shape[1]):
    x_max[i] = np.max(np.abs(X_data[:,i]))
    if np.abs(x_max[i]) > 0: 
        X_data[:,i] = X_data[:,i]/x_max[i]
    else:
        pass

cond_max = np.max(np.abs(y_data))
if np.abs(cond_max) > 0:
    y_data = y_data/cond_max
else:
    pass


In [28]:
y_data.shape

(878648,)

In [42]:
def generate_cond_data(n_model,stats,y_samples):
    z_samples = np.empty([stats,latent_dim])

    mean = list_of_encoders[n_model].predict([X_data[:trainsize], y_data[:trainsize]])[0]
    logvar = list_of_encoders[n_model].predict([X_data[:trainsize], y_data[:trainsize]])[1]
    var = np.exp(logvar)
    std = np.sqrt(var)

    l=0
    for i in range(0,stats):
        for j in range(0,latent_dim):
            z_samples[l,j] = np.random.normal(mean[i%trainsize,j], 0.05+std[i%trainsize,j])
        l=l+1
    new_events = list_of_decoders[n_model].predict([z_samples, y_samples])
    
    return new_events

In [43]:
nEvent_per_model = trainsize
fake_SRs = []
for i in range(len(list_of_decoders)):

    fake_SR = generate_cond_data(i,nEvent_per_model,y_data[:trainsize])
    print(fake_SR.shape[0])

    for i in range(0,fake_SR.shape[1]):
    	fake_SR[:,i]=fake_SR[:,i]*x_max[i]

    fake_SRs.append(fake_SR)

15622/15622 [==============================] - 17s 1ms/step
499889


In [51]:

plotting_ranges = [(0,0.8), (0,1), (0,1), (0,1), (0,1), (0,1)]
variables_names =  train_features
vnames = train_features

def fake_vs_real(gen_data,real_data,doLog,names):
    
    plt.figure(figsize=(20,15))
    colors = ["steelblue", "darkturquoise"]
    for p in range(0,nFeat):
        R = np.linspace(plotting_ranges[p][0],plotting_ranges[p][1],51)
        plt.subplot((nFeat+1)//2, 2, p+1)
        for rd in real_data: b_hist_y, b_hist_x, _ = plt.hist(rd[:,p], R, color="black", histtype='step', linewidth=2, label="SR Bkg", density=True)        
        for i,n in enumerate(gen_data): plt.hist(n[:,p], R, color=colors[i], histtype='stepfilled', alpha=0.5, linewidth=2, label="Generated Bkg %s"%names[i], density=True)
        plt.hist(X_data_unscaled[:,p], R, color="red", histtype='step', linewidth=2, label="SB Bkg", density=True)
        # s_hist_y, s_hist_x, _ = plt.hist(sig_data[:,p], R, color="r", histtype='step', linewidth=3, label="Signal", density=True)
        plt.xlabel("%s"%variables_names[p])
        if doLog == True: plt.yscale('log')
        plt.ylabel("Number of Events / bin")
        plt.legend(fontsize="large")
        ymax_b = np.max(b_hist_y)
        # ymax_s = np.max(s_hist_y)
        plt.ylim(0,ymax_b*1.5)
#     plt.savefig("fake_gen_compare_cbvae_6var_dynamicbeta.png")

/tmp/ipykernel_34388/3004731836.py:22: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  plt.ylim(0,ymax_b*1.5)


In [52]:
fake_vs_real([np.reshape(fake_SRs, (trainsize,nFeat))],[X_innerdata_train],1,[""])